# SERVER CODE

# Steps to Run the Client Server on your local machine

- 1. Run the Server Notebook
- 2. Run the Client Notebook
- 3. On the Server Notebook. Use the command "all" to connect your server to the client.
- 4. Use command "game" to run the game.
- 5. Use the command "win" to view the score

In [1]:
import random 
from functools import reduce
from collections import defaultdict
import numpy as np
from copy import copy
SUIT = ['H','S','D','C']
RANK = ['A', '2', '3', '4', '5','6','7']
RANK_VALUE = {'A': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'T': 10, 'Q': 10, 'K': 10}

class Card :
    def __init__(self,rank,suit):
        self.rank = rank
        self.suit = suit
        self.rank_to_val = RANK_VALUE[self.rank]
    
    def __str__(self):
        return f'{self.rank}{self.suit}'

    def __repr__(self):
        return f'{self.rank}{self.suit}'

    
    def __eq__(self,other):
        return self.rank == other.rank and self.suit == other.suit

# Deck class contains some basic operations performed with the cards:
# 1. Shuffling the cards.
# 2. Drawing card from the deck.
    
class Deck:
    def __init__(self,packs):
        self.packs = packs
        self.cards = []
        for pack in range(0,packs) :
            for suit in SUIT :
                for rank in RANK :
                    self.cards.append(Card(rank,suit))
    
    def shuffle(self):
        random.shuffle(self.cards)
    
    def draw_card(self):
        card = self.cards[0]
        self.cards.pop(0)
        return card

class Player:
  
    """  
        Player class to create a player object.
        eg: player = Player("player1", list(), isBot = False)
        Above declaration will be for your agent.
        All the player names should be unique or else you will get error.
        
    """

    def __init__(self,name,stash=list(),isBot=False,points=0,conn=None):
        self.stash = stash
        self.name = name
        self.game = None
        self.isBot = isBot
        self.points = points
        self.conn = conn
        
    def deal_card(self,card):
        try :
            self.stash.append(card)
            if len(stash) > self.game.cardsLength + 1 :
                raise ValueError('Cannot have cards greater than ')
        except ValueError as err:
            print(err.args)
            
    def drop_card(self,card):
        self.stash.remove(card)
        self.game.add_pile(card)
        return -1

    
    def meld(self):
        card_hash = defaultdict(list)
        for card in self.stash:
            card_hash[card.rank].append(card)
        melded_card_ranks = []
        for (card_rank,meld_cards) in card_hash.items():
            if len(meld_cards) >= 3 :
                self.game.meld.append(meld_cards)
                melded_card_ranks.append(card_rank)
                for card in meld_cards:
                    self.stash.remove(card)
        
        for card_rank in melded_card_ranks :
            card_hash.pop(card_rank)
        return len(melded_card_ranks) > 0

    def stash_score(self) :
        score = 0
        for card in self.stash :
            score += RANK_VALUE[card.rank]
        return score
    
    def get_info(self,debug):
        if debug :
            print(f'Player Name : {self.name} \n Stash Score: {self.stash_score()} \n Stash : {", ".join(str(x) for x in self.stash)}')
        card_ranks = []
        card_suits = []
        pileset = None
        pile = None
        for card in self.stash :
          card_suits.append(RANK_VALUE[card.rank])
          card_ranks.append(card.suit)
        if len(self.game.pile) > 0 : 
            return {"Stash Score" : self.stash_score(), "CardSuit":  card_suits, "CardRanks": card_ranks, "PileRank": self.game.pile[-1].rank, "PileSuit":self.game.pile[-1].suit}
        return {"Stash Score" : self.stash_score(), "CardSuit":  card_suits, "CardRanks": card_ranks}

class RummyAgent() :
    """
    Simple Rummy Environment
    
    Simple Rummy is a game where you need to make all the cards in your hand same before your opponent does.
    Here you are given 3 cards in your hand/stash to play.
    For the first move you have to pick a card from the deck or from the pile. 
    The card in deck would be random but you can see the card from the pile.
    In the next move you will have to drop a card from your hand.
    Your goal is to collect all the cards of the same rank. 
    Higher the rank of the card, the higher points you lose in the game. 
    You need to keep the stash score low. Eg, if you can AH,7S,5D your strategy would be to either find the first pair of the card or by removing the highest card in the deck.
    You only have 20 turns to either win the same or collect low scoring card.
    You can't see other players cards or their stash scores.
    
    Parameters
    ====
    players: Player objects which will play the game.
    max_card_length : Number of cards each player can have
    max_turns: Number of turns in a rummy game
    """

    def __init__(self,players,max_card_length=5,max_turns=20) :
        self.max_card_length = max_card_length
        self.max_turns = max_turns
        self.reset(players)
        
    def update_player_cards(self,players):
        for player in players :
            player = Player(player.name,list(),isBot=player.isBot,points=player.points,conn=player.conn)
            stash = []
            for i in range(self.max_card_length):
                player.stash.append(self.deck.draw_card())
            player.game = self
            self.players.append(player)
        self.pile = [self.deck.draw_card()]

    def add_pile(self,card):
        if len(self.deck.cards) == 0 :
            self.deck.cards.extend(self.pile)
            self.deck.shuffle()
            self.pile = []
        self.pile.append(card)
        
        
    def pick_card(self,player,action):
        if action == 0:
            self.pick_from_pile(player)
        else :
            self.pick_from_deck(player)
        if player.meld() :
            return {"reward" : 10}
        else :
            return {"reward" : -1}
#             return -player.stash_score()
       
    def pick_from_pile(self,player):
        card = self.pile[-1]
        self.pile.pop()
        return player.stash.append(card)
     
    def pick_from_deck(self,player):
        return player.stash.append(self.deck.draw_card())
    
    def get_player(self,player_name):
        return_player = [player for player in self.players if player.name == player_name]
        if len(return_player) != 1:
            print("Invalid Player")
            return None
        else:
            return return_player[0]
    
    def drop_card(self,player,card):
        player.drop_card(card)
        return {"reward" : -1}
    
  
    def computer_play(self,player):
        #Gets a card from deck or pile
        if random.randint(0,1) == 1 :
            self.pick_from_pile(player)
        else :
            self.pick_from_deck(player)
            
        #tries to meld if it can
#         if random.randint(0,10) > 5 :
        player.meld()
        
        #removes a card from the stash
        if len(player.stash) != 0:
            card = player.stash[(random.randint(0,len(player.stash) - 1))]
            player.drop_card(card)
        
    def play(self):
        for player in self.players :
            if len(player.stash) == 0 :
                return True
        if self.max_turns <= 0 :
            return True
        return False
    
    def _update_turn(self):
        self.max_turns -= 1
                  
    def reset(self,players,max_turns=20):
        self.players = []
        self.deck = Deck(1)
        self.deck.shuffle()
        self.max_turns = max_turns
        self.meld = []
        self.pile = []
        self.update_player_cards(players)



In [ ]:
import socket
import sys
import time
import threading
import time
import json
from queue import Queue
import pdb
NUMBER_OF_THREADS = 2
JOB_NUMBER = [1, 2]
queue = Queue()
all_connections = []
all_address = []
scores=[]
median=0
count=[0]
# TO BE CHANGED BEFORE COMPETITION
names=[[],[]]

# Create a Socket ( connect two computers)
def create_socket():
    try:
        global host
        global port
        global s
        host = ""
        port = 9999
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        s.setblocking(1)
    except socket.error as msg:
        print("Socket creation error: " + str(msg))


# Binding the socket and listening for connections
def bind_socket():
    try:
        global host
        global port
        global s
        print("Binding the Port: " + str(port))

        s.bind((host, port))
        s.listen(5)

    except socket.error as msg:
        print("Socket Binding error" + str(msg) + "\n" + "Retrying...")
        bind_socket()


# Handling connection from multiple clients and saving to a list
# Closing previous connections when server.py file is restarted
count=[0]
def accepting_connections():
    for c in all_connections:
        c.close()

    del all_connections[:]
    del all_address[:]

    while True:
        try:
            conn, address = s.accept()
            s.setblocking(1)  # prevents timeout

            all_connections.append(conn)
            all_address.append(address)

            print("Connection has been established :" + address[0])
            c=count[0]
            c=c+1
            print(c)
            count[:] = []
            count.append(c)
        except:
            print("Error accepting connections")


# 2nd thread functions - 1) See all the clients 2) Select a client 3) Send commands to the connected client
# Interactive prompt for sending commands
# turtle> list
# 0 Friend-A Port
# 1 Friend-B Port
# 2 Friend-C Port
# turtle> select 1
# 192.168.0.112> dir


def start_turtle():
    while True:
        cmd = input('turtle> ')
        if cmd == 'list':
            list_connections()
        elif 'all' in cmd:
            start_collect()
        elif 'game' in cmd:
            start_result()
        elif 'win' in cmd:
            start_win()
        elif 'select' in cmd:
            conn = get_target(cmd)
            if conn is not None:
                send_target_commands(conn)

        else:
            print("Command not recognized")


# Display all current active connections with client
def start_win():
    names[1], names[0] = zip(*sorted(zip(names[1], names[0])))
    k=len(names[0])
#     pdb.set_trace()
    for i in range(k) :
        print(f'Player : {names[0][i]} , Score: {names[1][i]}')
    # TO BE CHANGED BEFORE COMPETITION
    # k>36 70-1 k-36
#     if k>3:
#         for i in range(k-1,k-10,-1):
#             print(22-i," Player ",names[0][i]," Score ",names[1][i])
    
def start_collect():
    f=len(all_connections)
    for i in range (0,f):
        
        cmd=i
        conn = get_target(cmd)
        if conn is not None:
                send_target_commands(conn)
    print ("Collected all names")   
        
def start_result():
    f=len(all_connections)
    #To be changed to select number of players
    step_size = 1
    for i in range (0,f,step_size):
        cmd=i
        conns = []
        grp_names = []
        if (i+step_size - 1) < f :
            for j in range(i,i+step_size ):
                conns.append(get_target(j))
                grp_names.append(names[0][j])
        if len(conns) > 0:
            points=startgame(conns,grp_names)
            for point in points :
                names[1].append(point)
            print(names[1])
    print ("All Games played")
    
def list_connections():
    results = ''

    for i, conn in enumerate(all_connections):
        try:
            conn.send(str.encode(' '))
            conn.recv(20480)
        except:
            del all_connections[i]
            del all_address[i]
            continue

        results = str(i) + "   " + str(all_address[i][0]) + "   " + str(all_address[i][1]) + "\n"

    print("----Clients----" + "\n" + results)


# Selecting the target
def get_target(cmd):
    try:
        #target = cmd.replace('select ', '')  # target = id
        target = cmd#int(target)
        conn = all_connections[target]
        print("You are now connected to :" + str(all_address[target][0]))
        print(str(all_address[target][0]) + ">", end="")
        return conn
        # 192.168.0.4> dir

    except:
        print("Selection not valid")
        return None


# Send commands to client/victim or a friend
def send_target_commands(conn):
    while True:
        try:
            cmd = 'send'
            if cmd == 'quit':
                break
            if len(str.encode(cmd)) > 0:
                conn.send(str.encode(cmd))
                client_response = str(conn.recv(20480), "utf-8")
                print(client_response, end="")
                print('\n')
                names[0].append(client_response)
                
                print(names[0])
                
                conn.send(str.encode("Connection established, Names collected"))
                break
        except:
            print("Error sending commands")
            break
            
def startgame(conns,players):
    grp_players = []
    for conn,player in zip(conns,players) :
        grp_players.append(Player(player,list(),conn=conn))
        print('game started for ',player)
    if len(conns) == 1 :
        grp_players.append(Player('comp1',list(),isBot=True))
#     p1 = Player(player,list(),)
#     p2 = Player('comp1',list(),isBot=True)
    env = RummyAgent(grp_players,max_card_length=3,max_turns=20)
    #Number of games
    number_of_games = 5
    
    #Number of rounds
    number_of_rounds = env.max_turns
    
    debug = True
    for _game in range(number_of_games):
        env.reset(env.players)
        random.shuffle(env.players)
        _round = 0 
        print("-"* 50)
        print("Game Number: {}".format(_game + 1))
        print("-"* 50)
        while not env.play():
            env._update_turn()
            print("%"* 50)
            print("Game Number: {} Round Number: {}".format(_game+1, _round+1))
            print("%"* 50)
            _round += 1
            
            for player in env.players:
                if player.isBot :
                    if env.play():
                        continue
                    if debug :
                        print(f'{player.name} Plays')
                    env.computer_play(player)
                    if debug :
                        player.get_info(debug)
                        if player.stash == 0 :
                            print(f'{player.name} wins the round')
                else :
                    if env.play() :
                        continue
                    round_obs = player.get_info(debug)
                    json.dumps(round_obs)
                    #Send The status
#                     pdb.set_trace()
                    player.conn.send(str.encode(f'sendinfo'))
                    ro=str(player.conn.recv(20480), "utf-8")
                    print(ro)
                    player.conn.send(str.encode(str(round_obs)))
                    #Wait to receive it
                    
                    ro=str(player.conn.recv(20480), "utf-8")
                    ro=int(ro)
                    print(f'Action pick : {ro}')
                    env.pick_card(player,ro)
                    print("Send action for card drop")
                    round_obs = player.get_info(debug)
                    json.dumps(round_obs)
                    player.conn.send(str.encode(f'sendaction_for_card_drop'))
                    ro=str(player.conn.recv(20480), "utf-8")
                    print(ro)
                    player.conn.send(str.encode(str(round_obs)))
                    ro=str(player.conn.recv(20480), "utf-8")
                    ro=int(ro)%10
                    print(f'Action Drop {ro}')
                    if len(player.stash) == 1:
                        env.drop_card(player,player.stash[0])
                    else :
                        env.drop_card(player,player.stash[ro])

        
        result = ''
        for player in env.players:
            if player.conn != None :
                player.conn.send(str.encode('round_over'))
                ro = str(player.conn.recv(20480),"utf-8")
                print(ro)
            player.points += player.stash_score()
            res = f'Player: {player.name}  Score: {player.points} Stash Score : {player.stash_score()}\n'
            print(res)
            result += res 
        for player in env.players : 
            if player.conn != None :
                player.conn.send(str.encode(result))
    points = []
    for _player in env.players:
        print(f'Name: {_player.name} Score: {_player.points} ' )
#         if _player.name == player:
#             points = _player.points
    for player in env.players :
        if player.conn != None :
            player.conn.send(str.encode('gameover'))
            points.append(player.points)
    return points



# Create worker threads
def create_workers():
    for _ in range(NUMBER_OF_THREADS):
        t = threading.Thread(target=work)
        t.daemon = True
        t.start()


# Do next job that is in the queue (handle connections, send commands)
def work():
    while True:
        x = queue.get()
        if x == 1:
            create_socket()
            bind_socket()
            accepting_connections()
        if x == 2:
            start_turtle()

        queue.task_done()


def create_jobs():
    for x in JOB_NUMBER:
        queue.put(x)

    queue.join()


create_workers()
create_jobs()

Binding the Port: 9999
